<img align="right" src="http://admit.usc.edu/images/usc-logo.jpg" width="350px" height="350px"/>
<h1 style="font-size:20pt">SAGE Project | Web Scraping </h1><br/>
<b> Author: </b> Vyshali Badanidyoor<br/>
<b> Created: </b>13 December 2020<br/>
<b>Version: </b>v20.01.02<br/>

## Table of Contents:

* [1. Import Required Packages](#import-packages-1)
* [2. Define Page URL](#define-url)
* [3. Extract page content and parse HTML](#extract-page-content)
* [4. Data Manipulation](#data-manipulation)
* [5. References](#references)

### 1. Import required packages <a class="anchor" id="import-packages-1"></a>

In [37]:
# Module to convert dates into specific formats
try:
    import dateutil.parser as parser
    from datetime import timedelta, date
except:
    print('Failed to load requirement - dateutil.parser')

# Library to make HTTP requests
try:
    import requests
except:
    print('Failed to load requirement - requests')

# Package to parse HTML and XML webpages
try:
    from bs4 import BeautifulSoup
except:
    print('Failed to load requirement - Beautiful Soup')

# library for data manipulation and analysis
try:
    import pandas as pd
except:
    print('Failed to load requirement - pandas')

### 2. Define Page URL<a class="anchor" id="define-url"></a>

In [38]:
pageUrl = 'https://en.wikipedia.org/wiki/2019_in_spaceflight#Orbital_launches'

### 3. Extract page content and parse HTML <a class="anchor" id="extract-page-content"></a>

In [39]:
# Extract page content
response = requests.get(pageUrl)
html = response.content

In [40]:
# Convert HTML to beautifulsoup format for parsing
soup = BeautifulSoup(html, 'html.parser')

#### 3.1 Find class selector of table of interest

In [41]:
# Extract 'Orbital Launches' table content using class selector 'wikitable collapsible'
tableOuput = soup.find('table',{'class':'wikitable collapsible'})

#### 3.2 Initialize variables

In [42]:
# Main iterator variable for every row in 'Orbital Launches' table
i=4

# List variable to hold date and launch status for each orbital launch in 2019
orbitalLaunchlist = []

# Variable to track number of payloads corresponding to each launch date and time
rowspan=0

# List to define criteria of successful launch based on launch status
successList = ['Successful', 'Operational', 'En Route']

#### 3.3 Logic to get date and launch status for each orbital launch in 2019

In [43]:
while i !=0:
    
    # Make 'Orbital Launches' table content accessible inside while loop
    global tableOutput
    
    # Make 'successList' variable accessible inside loop
    global successList
    
    # List variable to hold status of each launch
    rowSubStatusList = []
    
    # Extract content of one row in 'Orbital Launches' table
    try:
        row = tableOuput.findAll('tr')[i]
    # Break out of loop when end of table is reached
    except:
        break
    
    # Extract colspan of row
    colspan = row.find('td').get("colspan")
    
    # Logic to skip rows that provide navigation to different months of the year
    if colspan is not None and int(colspan)==7:
        # Increment iterator to point to row that has actual launch details
        i=i+2
        # Extract row contents
        row = tableOuput.findAll('tr')[i]
    
    # Extract date of orbital launch
    try:
        rowDate = row.find('td').find('span').contents[0]+ ' 2019 '
    except:
        rowDate = ''
    
    # Extract rowspan variable to track number of payloads corresponding to each launch date and time
    try:
        rowspan = row.find('td').get("rowspan")
    except:
        rowspan = 'None'
    
    # Loop to iterate over payloads corresponding to each launch date and time
    for j in range(1,int(rowspan)):
        
        # Extract row content of each payload
        try:
            rowSub = tableOuput.findAll('tr')[i+j]
        except:
            rowSub = ''
        
        # Extract status of launch for each payload and append to list
        try:
            rowSubStatus = rowSub.findAll('td')[5].contents[0].strip()
            rowSubStatusList.append(rowSubStatus)
        except:
            rowSubStatus = 'None'
        
        # If payload launch was successful, then break from second loop and go to next launch date-time
        if rowSubStatus in successList: 
            break
        # If payload launch was unsuccessful, then repeat steps above (in inner loop) and keep only status of successful launch (if any)
        else:
            rowSubStatusList = [x for x in rowSubStatusList if x in successList]
            
    # Increment main iterator to point to next launch date-time
    if rowspan is not None:
        i = i+int(rowspan)
    
    # Append launch date and status to 'orbitalLaunchlist'
    orbitalLaunchlist.append([rowDate, rowSubStatusList])

In [44]:
# Preview list contents
orbitalLaunchlist[:10]

[['10 January 2019 ', ['Operational']],
 ['11 January 2019 ', ['Operational']],
 ['15 January 2019 ', []],
 ['18 January 2019 ', ['Operational']],
 ['19 January 2019 ', ['Operational']],
 ['21 January 2019 ', ['Operational']],
 ['24 January 2019 ', ['Operational']],
 ['5 February 2019 ', []],
 ['5 February 2019 ', ['Operational']],
 ['21 February 2019 ', ['Operational']]]

### 4. Data Manipulation<a class="anchor" id="data-manipulation"></a>

In [45]:
# Convert list to dataframe and rename columns
orbitalLaunchDf = pd.DataFrame(orbitalLaunchlist, columns = ['date' , 'LaunchStatus'])

In [46]:
# Preview dataframe
orbitalLaunchDf.head()

,date,LaunchStatus
0,10 January 2019,[Operational]
1,11 January 2019,[Operational]
2,15 January 2019,[]
3,18 January 2019,[Operational]
4,19 January 2019,[Operational]


In [47]:
orbitalLaunchDf['LaunchStatus'] = orbitalLaunchDf['LaunchStatus'].str[0]

In [48]:
# Replace launch status with numeric values
orbitalLaunchDf['LaunchStatus'].replace(['Operational', 'Successful', 'En route'], 1, inplace=True)
orbitalLaunchDf['LaunchStatus'].fillna(0, inplace=True)

In [49]:
# Rename columns
orbitalLaunchDf.rename(columns={"LaunchStatus": "value"}, inplace=True)

In [50]:
# Convert to Pandas datetime format for date manipulation
orbitalLaunchDf['date'] = pd.to_datetime(orbitalLaunchDf['date'])

In [51]:
# Aggregate launches based on date
orbitalLaunchDf = orbitalLaunchDf.groupby(by='date').agg({'value': 'sum'}).reset_index()

In [52]:
# Preview manipulated dataframe
orbitalLaunchDf.head()

,date,value
0,2019-01-10,1.0
1,2019-01-11,1.0
2,2019-01-15,0.0
3,2019-01-18,1.0
4,2019-01-19,1.0


In [53]:
# Include all dates in 2019
def getDatesInDateRange(start, end):
    for n in range(int ((end - start).days)+1):
        yield start + timedelta(n)

# Define start and end dates of 2019
startDate = date(2019, 1, 1)
endDate = date(2019, 12, 31)

# Initialize list to contain all dates in 2019
dateRangeList = []

# Iterate over each date in date range specified
for dt in getDatesInDateRange(startDate, endDate):
    dateRangeList.append(dt.strftime("%Y-%m-%d"))
    
# Convert list to dataframe
dateRangeDf = pd.DataFrame(dateRangeList, columns=['date'])

# Convert string to datetime object
dateRangeDf['date'] = pd.to_datetime(dateRangeDf['date'])

In [54]:
# Preview dataframe
dateRangeDf.head()

,date
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05


In [55]:
# Aggregate launches based on date
finalOrbitalLaunchDf = pd.merge(dateRangeDf, orbitalLaunchDf, on='date', how='left')

# Fill a value of 0 for dates with no launches
finalOrbitalLaunchDf['value'].fillna(0, inplace=True)

In [56]:
# Coerce values from float64 to int64
finalOrbitalLaunchDf['value'] = finalOrbitalLaunchDf['value'].astype(int)

In [57]:
finalOrbitalLaunchDf.head(20)

,date,value
0,2019-01-01,0
1,2019-01-02,0
2,2019-01-03,0
3,2019-01-04,0
4,2019-01-05,0
5,2019-01-06,0
6,2019-01-07,0
7,2019-01-08,0
8,2019-01-09,0
9,2019-01-10,1


In [58]:
# Write dataframe to CSV
finalOrbitalLaunchDf.to_csv('_output.csv')

### 5. References <a class="anchor" id="refernces"></a>

* [Web Scraping Wiki Tables](https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722)<br>
* [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)<br>
* [Replace Dataframe Values](https://stackoverflow.com/questions/27060098/replacing-few-values-in-a-pandas-dataframe-column-with-another-value)<br>
* [Convert String to ISO8601 Format](https://stackoverflow.com/questions/4460698/python-convert-string-representation-of-date-to-iso-8601)<br>
* [Delete List Items](https://stackoverflow.com/questions/7623715/deleting-list-elements-based-on-condition)<br>
* [Get dates in a range](https://www.w3resource.com/python-exercises/date-time-exercise/python-date-time-exercise-50.php)<br>